
Deposition of integrative models<a id="mainpage"></a>
========================

**Table of contents**

 - [Introduction](#introduction)
 - [Why PDB-Dev?](#whypdbdev)
 - [Why mmCIF?](#whymmcif)
 - [Study versus single model](#studyvmodel)
 - [File format](#fileformat)
 - [Basic usage of ProtocolOutput](#basicusage)
 - [Linking to other data](#linking)
 - [Annotation of input files](#annotation)
 - [Polishing the deposition](#polishing)
  - [Cross-linker type](#xltype)
  - [Correct number of output models](#fixnummodel)
  - [Add model coordinates](#addcoords)
  - [Replace local links with DOIs](#adddois)
 - [Output](#output)
 - [Visualization](#visualization)
  - [ChimeraX](#chimerax)
  - [Web browser](#browser)
  - [Plain text](#plain)


# Introduction<a id="introduction"></a>

In this tutorial we will introduce the procedure used to deposit integrative modeling studies in the [PDB-Dev](https://pdb-dev.wwpdb.org/) database in mmCIF format.

We will demonstrate the procedure using [IMP](https://integrativemodeling.org/) and its PMI module, but the database will accept integrative models from any software, as long as they are compliant mmCIF files (e.g. there are several HADDOCK and Rosetta models already in PDB-Dev).

# Why PDB-Dev?<a id="whypdbdev"></a>

PDB-Dev is a database run by the wwPDB. It is specifically for the deposition of *integrative* models, i.e. models generated using more than one source of experimental data. (Models that use only one experiment generally go in PDB; models that use no experimental information - theoretical models - go in [ModelArchive](https://www.modelarchive.org/)).

# Why mmCIF?<a id="whymmcif"></a>

wwPDB already uses the mmCIF file format for X-ray structures, and has a formal data model ([PDBx](http://mmcif.wwpdb.org/)) to describe these structures. The format is *extensible*; extension dictionaries exist to describe NMR, SAS, EM data, etc. For integrative models, we use PDBx plus an "integrative/hybrid methods" (IHM) [extension dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Index/). This supports coarse-grained structures, multiple input experimental data sources, multiple states, multiple scales, and ensembles related by time or other order.

# Study versus single model<a id="studyvmodel"></a>

A frequently asked question is whether mmCIF files can be generated directly from an [IMP::Model](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1Model.html) or an
[RMF file](https://integrativemodeling.org/rmf/). This generally isn't possible because RMF and [IMP::Model](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1Model.html) are designed to store one or more output models, where each model is a set of coordinates for a single conformation of the system being studied. A deposition, on the other hand, aims to cover a complete modeling study, and should capture the entire ensemble of output models, all of the input data needed to reproduce the modeling, and quality metrics such as the precision of the ensemble and the degree to which it fits the output data. A deposition is also designed to
be visualized, and so may contain additional data not used in the modeling itself, such as preset colors or views to match figures in the publication or highlight regions of interest, and more human-descriptive names for parts
of the system. Thus, deposition is largely a data-gathering exercise, and benefits from a modeling study being tidy and well organized (for example, by storing it in a [GitHub](https://github.com) repository) so that data is easy to find and track to its source.

# File format<a id="fileformat"></a>

The file format used by PDB for deposition of integrative modeling studies is the same [mmCIF format](http://mmcif.wwpdb.org/) that is used for crystal structures, with extensions (the [Integrative/Hybrid Methods (IHM) dictionary](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Index/)) to support coarse-grained structures, multiple input experimental data sources, multiple states, multiple scales, and ensembles related by time or other order. Currently, models that are compliant with the IHM dictionary can be deposited in the [PDB-Dev](https://pdb-dev.wwpdb.org/) database.

mmCIF is a text format with a well-defined syntax, so in principle files could be generated by hand or with simple scripts. However, it is generally easier to use the existing [python-ihm](https://github.com/ihmwg/python-ihm)
library. This stores the same data as in an mmCIF file, but represents it as a set of Python classes, so it is easier to manipulate. IMP includes a copy of the python-ihm library, and uses it internally to read and write\ mmCIF files.

There are two main approaches to generating an mmCIF file for deposition:

 1. Use the [python-ihm](https://github.com/ihmwg/python-ihm) library directly, by writing a Python script that reads in output models and input data, adds annotations, and writes out an mmCIF file.
 2. Use the [ProtocolOutput](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html) class to automatically capture an entire [IMP::pmi](https://integrativemodeling.org/2.12.0/doc/ref/namespaceIMP_1_1pmi.html) modeling protocol.

The first approach offers the greatest flexibility for custom protocols or for modeling outside of IMP itself, but does require everything to be done manually. For a good example of this approach, see the [modeling of Nup133](https://github.com/integrativemodeling/nup133/tree/master/outputs_foxs_ensemble_new/pdb-dev). (One example of the tradeoff between flexibility and manual coding: the cross-links used in the study are stored in a [plain text file](https://github.com/integrativemodeling/nup133/blob/master/Crosslinks/DSS_EDC_crosslinks.txt) so a [custom Python class](https://github.com/integrativemodeling/nup133/blob/master/outputs_foxs_ensemble_new/pdb-dev/xlink.py) had to be written to parse them.)

The second approach will be explored in this tutorial.

# Basic usage of ProtocolOutput<a id="basicusage"></a>

[ProtocolOutput](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html) is designed to be attached to a top-level PMI object (usually [IMP::pmi::topology::System](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1pmi_1_1topology_1_1System.html)). Then, as the script is run, it will capture all of the information about the modeling
study, in an [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object. Additional information not in the modeling script itself, such as the
resulting publication, can be added using the [python-ihm API](https://python-ihm.readthedocs.io/en/latest/usage.html).

We will demonstrate ProtocolOutput by using it to make a deposition of the RNA Pol II stalk modeling, as covered in the [introductory PMI tutorial](https://integrativemodeling.org/tutorials/rnapolii_stalk/). First, download the files for this tutorial by using the "Clone or download" link at the [tutorial's GitHub page](https://github.com/salilab/imp_deposition_tutorial/). We will then modify the `modeling.py` script from that tutorial, calling it [deposition.py script](https://github.com/salilab/imp_deposition_tutorial/blob/master/rnapolii/modeling/deposition.py) in the `rnapolii/modeling` subdirectory, to capture the study as mmCIF.

The first modification to `deposition.py` is to import the PMI mmCIF and python-ihm Python modules:

In [ ]:
import IMP.pmi.mmcif
import ihm

In [ ]:

import IMP
import IMP.core
import IMP.algebra
import IMP.atom
import IMP.container

import IMP.pmi.restraints.crosslinking
import IMP.pmi.restraints.stereochemistry
import IMP.pmi.restraints.em
import IMP.pmi.restraints.basic
import IMP.pmi.tools
import IMP.pmi.samplers
import IMP.pmi.output
import IMP.pmi.macros
import IMP.pmi.topology

import os
import sys


#---------------------------
# Define Input Files
#---------------------------
datadirectory = "../data/"
topology_file = datadirectory+"topology.txt"
target_gmm_file = datadirectory+'emd_1883.map.mrc.gmm.50.txt'

# Initialize model
m = IMP.Model()

# Read in the topology file.
# Specify the directory wheere the PDB files, fasta files and GMM files are
topology = IMP.pmi.topology.TopologyReader(topology_file,
                                  pdb_dir=datadirectory,
                                  fasta_dir=datadirectory,
                                  gmm_dir=datadirectory)

Next, as soon as the top-level [IMP::pmi::topology::System](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1pmi_1_1topology_1_1System.html) object is created, we attach a ProtocolOutput object:

In [ ]:
# Use the BuildSystem macro to build states from the topology file
bs = IMP.pmi.macros.BuildSystem(m)

# Record the modeling protocol to an mmCIF file
po = IMP.pmi.mmcif.ProtocolOutput(None)
bs.system.add_protocol_output(po)
po.system.title = "Modeling of RNA Pol II"
# Add publication
po.system.citations.append(ihm.Citation.from_pubmed_id(25161197))

Note that the ProtocolOutput object `po` simply wraps an [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) object as `po.system`. We can then customize the [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) by setting a human-readable title and adding a citation (here we use
[ihm.Citation.from_pubmed_id](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.Citation.from_pubmed_id), which looks up a citation by PubMed ID - this particular PubMed ID is
actually for the previously-published [modeling of the Nup84 complex](https://salilab.org/nup84/)).

In [ ]:

# Each state can be specified by a topology file.
bs.add_state(topology)

# Build the system representation and degrees of freedom
root_hier, dof = bs.execute_macro(max_rb_trans=4.0,
                                  max_rb_rot=0.3,
                                  max_bead_trans=4.0,
                                  max_srb_trans=4.0,
                                  max_srb_rot=0.3)

# Fix all rigid bodies but not Rpb4 and Rpb7 (the stalk)
# First select and gather all particles to fix.
fixed_particles=[]
for prot in ["Rpb1","Rpb2","Rpb3","Rpb5","Rpb6","Rpb8","Rpb9","Rpb10","Rpb11","Rpb12"]:
    fixed_particles+=IMP.atom.Selection(root_hier,molecule=prot).get_selected_particles()

# Fix the Corresponding Rigid movers and Super Rigid Body movers using dof
# The flexible beads will still be flexible (fixed_beads is an empty list)!
fixed_beads,fixed_rbs=dof.disable_movers(fixed_particles,
                                         [IMP.core.RigidBodyMover,
                                          IMP.pmi.TransformMover])

# Randomize the initial configuration before sampling, of only the molecules
# we are interested in (Rpb4 and Rpb7)
IMP.pmi.tools.shuffle_configuration(root_hier,
                                    excluded_rigid_bodies=fixed_rbs,
                                    max_translation=50,
                                    verbose=False,
                                    cutoff=5.0,
                                    niterations=100)

outputobjects = [] # reporter objects (for stat files)

#-----------------------------------
# Define Scoring Function Components
#-----------------------------------

# Here we are defining a number of restraints on our system.
#  For all of them we call add_to_model() so they are incorporated into scoring
#  We also add them to the outputobjects list, so they are reported in stat files

# Connectivity keeps things connected along the backbone (ignores if inside
# same rigid body)
mols = IMP.pmi.tools.get_molecules(root_hier)
for mol in mols:
    molname=mol.get_name()
    IMP.pmi.tools.display_bonds(mol)
    cr = IMP.pmi.restraints.stereochemistry.ConnectivityRestraint(mol,scale=2.0)
    cr.add_to_model()
    cr.set_label(molname)
    outputobjects.append(cr)

# Excluded Volume Restraint
#  To speed up this expensive restraint, we operate it at resolution 20
ev = IMP.pmi.restraints.stereochemistry.ExcludedVolumeSphere(
                                         included_objects=root_hier,
                                         resolution=10)
ev.add_to_model()
outputobjects.append(ev)


# Crosslinks - dataset 1
#  To use this restraint we have to first define the data format
#  Here assuming that it's a CSV file with column names that may need to change
#  Other options include the linker length and the slope (for nudging components together)
xldbkwc = IMP.pmi.io.crosslink.CrossLinkDataBaseKeywordsConverter()
xldbkwc.set_protein1_key("pep1.accession")
xldbkwc.set_protein2_key("pep2.accession")
xldbkwc.set_residue1_key("pep1.xlinked_aa")
xldbkwc.set_residue2_key("pep2.xlinked_aa")

xl1db = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)
xl1db.create_set_from_file(datadirectory+'polii_xlinks.csv')

xl1 = IMP.pmi.restraints.crosslinking.CrossLinkingMassSpectrometryRestraint(
                                   root_hier=root_hier,
                                   CrossLinkDataBase=xl1db,
                                   length=21.0,
                                   slope=0.01,
                                   resolution=1.0,
                                   label="Trnka",
                                   weight=1.)

xl1.add_to_model()             # crosslink must be added to the model
outputobjects.append(xl1)


# Crosslinks - dataset 2
#  We can easily add a second set of crosslinks.
#  These have a different format and label, but other settings are the same
xldbkwc = IMP.pmi.io.crosslink.CrossLinkDataBaseKeywordsConverter()
xldbkwc.set_protein1_key("prot1")
xldbkwc.set_protein2_key("prot2")
xldbkwc.set_residue1_key("res1")
xldbkwc.set_residue2_key("res2")

xl2db = IMP.pmi.io.crosslink.CrossLinkDataBase(xldbkwc)
xl2db.create_set_from_file(datadirectory+'polii_juri.csv')

xl2 = IMP.pmi.restraints.crosslinking.CrossLinkingMassSpectrometryRestraint(
                                   root_hier=root_hier,
                                   CrossLinkDataBase=xl2db,
                                   length=21.0,
                                   slope=0.01,
                                   resolution=1.0,
                                   label="Chen",
                                   weight=1.)
xl2.add_to_model()
outputobjects.append(xl2)


# Electron Microscopy Restraint
#  The GaussianEMRestraint uses a density overlap function to compare model to data
#   First the EM map is approximated with a Gaussian Mixture Model (done separately)
#   Second, the components of the model are represented with Gaussians (forming the model GMM)
#   Other options: scale_to_target_mass ensures the total mass of model and map are identical
#                  slope: nudge model closer to map when far away
#                  weight: experimental, needed becaues the EM restraint is quasi-Bayesian
em_components = IMP.pmi.tools.get_densities(root_hier)

gemt = IMP.pmi.restraints.em.GaussianEMRestraint(em_components,
                                                 target_gmm_file,
                                                 scale_target_to_mass=True,
                                                 slope=0.000001,
                                                 weight=80.0)
gemt.add_to_model()
outputobjects.append(gemt)

#--------------------------
# Monte-Carlo Sampling
#--------------------------

#--------------------------
# Set MC Sampling Parameters
#--------------------------
num_frames = 20000
if '--test' in sys.argv: num_frames=100
num_mc_steps = 10

As it is generally undesirable to rerun the entire modeling just to generate an mmCIF file, we can save a lot of time by turning on `test_mode` which skips the actual Monte Carlo simulation:

In [ ]:
# This object defines all components to be sampled as well as the sampling protocol
mc1=IMP.pmi.macros.ReplicaExchange0(m,
                                    root_hier=root_hier,
                                    monte_carlo_sample_objects=dof.get_movers(),
                                    output_objects=outputobjects,
                                    crosslink_restraints=[xl1,xl2],    # allows XLs to be drawn in the RMF files
                                    monte_carlo_temperature=1.0,
                                    simulated_annealing=True,
                                    simulated_annealing_minimum_temperature=1.0,
                                    simulated_annealing_maximum_temperature=2.5,
                                    simulated_annealing_minimum_temperature_nframes=200,
                                    simulated_annealing_maximum_temperature_nframes=20,
                                    replica_exchange_minimum_temperature=1.0,
                                    replica_exchange_maximum_temperature=2.5,
                                    number_of_best_scoring_models=10,
                                    monte_carlo_steps=num_mc_steps,
                                    number_of_frames=num_frames,
                                    global_output_directory="output",
                                    test_mode=True)

# Start Sampling
mc1.execute_macro()

Once we're done with our PMI protocol, we call the [ProtocolOutput.finalize](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1pmi_1_1mmcif_1_1ProtocolOutput.html#a7b8395a33cbd3211cdf28e943c35dfa5) method to collect all of the information about the integrative modeling protocol in [ihm.System](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System):

In [ ]:
po.finalize()

At this point we have basic information which we can write out to an mmCIF file:

In [ ]:
import ihm.dumper
with open('initial.cif', 'w') as fh:
    ihm.dumper.write(fh, [po.system])

The `initial.cif` file can be viewed [in a text editor](#plain) or [with ChimeraX](#chimerax), or the `po.system`
object can be explored in a Python console. Each contains a great deal of information about the system, including:

 - the software (IMP and PMI) used in the modeling (in the `_software` table
   in the mmCIF file, or as [po.system.software](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.software) in Python).
 - cited papers (`_citation` table or [po.system.citations](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.citations) Python object).
 - the entities (unique polymer sequences) that make up the system
   (`_entity`, `_entity_poly`, `_entity_poly_seq` tables or [po.system.entities](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.entities)).
 - each asymmetric unit (instance of an entity) (`_struct_asym` table or [po.system.asym_units](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.asym_units)).
 - assemblies (collections of entities and/or asymmetric units that represent
   a subset of the system, such as everything modeled in a given state, or
   everything that fits into an electron microscopy density map) (`_ihm_struct_assembly` table or [po.system.complete_assembly](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.complete_assembly)).
 - input experimental information, such as crystal structures, comparative
   models, or cross-links (`_ihm_dataset_list` table or [ihm.dataset objects](https://python-ihm.readthedocs.io/en/latest/dataset.html#module-ihm.dataset)).
 - information about each restraint, such as cross-linking (`_ihm_cross_link_list` table or [ihm.restraint.CrossLinkRestraint objects](https://python-ihm.readthedocs.io/en/latest/restraint.html#ihm.restraint.CrossLinkRestraint))
   or fit to the electron microscopy density map (`_ihm_3dem_restraint` table or [ihm.restraint.EM3DRestraint objects](https://python-ihm.readthedocs.io/en/latest/restraint.html#ihm.restraint.EM3DRestraint)).
 - the Monte Carlo simulation (`_ihm_modeling_protocol` table or [ihm.protocol objects](https://python-ihm.readthedocs.io/en/latest/protocol.html#module-ihm.protocol)).

# Linking to other data<a id="linking"></a>

Integrative modeling draws on data from a variety of sources, so for a complete deposition all of this data needs to be available. The data is not placed directly in the mmCIF file - rather, the file contains links. These
links can be:

 - an identifier in a domain-specific database, such as
   [PDB](https://www.wwpdb.org/) or [EMDB](https://www.ebi.ac.uk/pdbe/emdb/).
 - a [DOI](https://www.doi.org/) where the files can be obtained.
 - a path to a file on the local disk.

Database identifiers are preferable because the databases are curated by domain experts and include domain-specific information, and the files are in standard formats. ProtocolOutput will attempt to use these where possible. For example, in this case ProtocolOutput is able to read the annotations of the input crystal structure used for the modeling ([see below](#annotation)) and determine that it is stored in the PDB, so the relevant [1WCM](https://www.rcsb.org/structure/1WCM) identitifer is included in the mmCIF file (see the `_ihm_dataset_related_db_reference` table). (Similarly, the EMDB [EMD-1883](https://www.ebi.ac.uk/pdbe/entry/emdb/EMD-1883) identifier is used for the EM density map.)

When a file is used for the modeling which cannot be tracked back to a database, ProtocolOutput will include its path (relative to that of the mmCIF file). For example, in this case the cross-links used are stored in simple CSV
files. In addition, the Python script itself is linked from the mmCIF file. Such local paths won't be available to end users, so for deposition we need to replace these paths with database IDs or DOIs ([see below](#polishing)).

As a further example of linkage, see the links in the previously-published [modeling of the Nup84 complex](https://salilab.org/nup84/) below. The mmCIF file links to the data directly used in the modeling (cross-links, crystal structures, electron microscopy class averages, comparative models, and Python scripts) via database IDs or DOIs. Furthermore, where available links are provided from this often-processed data back to the original data, such
as templates for comparative models, mass spectometry spectra for cross-links, or micrographs for class averages:

<img src="images/links.png" width="700px" title="Nup84 file linkage" />

# Annotation of input files<a id="annotation"></a>

ProtocolOutput will look at all input files to try to extract as much metadata
as possible. As [described above](#linking) this is used to look up database
identifiers, but it can also detect other inputs, such as the templates used
for comparative modeling. Thus, it is important for deposition that all input
files are annotated as well as possible:

 - deposit input files in a domain-specific database where possible and use
   the deposited file (which typically will contain identifying headers) in the
   modeling repository.
 - for PDB crystal structures, do not remove the original headers, such as
   the `HEADER` and `TITLE` lines.
 - for MODELLER comparative models, leave in the `REMARK` records and make sure
   that any files mentioned in `REMARK   6 ALIGNMENT:` or
   `REMARK   6 SCRIPT:` records are available (modify the paths if necessary,
   for example if you moved the PDB file into a different directory from the
   modeling script and alignment).
 - for manually generated PDB files, such as those extracted from a published
   work or generated by docking or other means, add suitable `EXPDTA` and
   `TITLE` records to the files for ProtocolOutput to pick up. See the
   [python-ihm docs](https://python-ihm.readthedocs.io/en/latest/metadata.html#ihm.metadata.PDBParser.parse_file)
   for more information.
 - for GMM files used for the EM density restraint, keep the original MRC
   file around and make sure that the `# data_fn:` header in the GMM file
   points to it.

# Polishing the deposition<a id="polishing"></a>

ProtocolOutput attempts to automatically generate as much as possible of the mmCIF file, but there are some areas where manual intervention is necessary because the data is missing, or its guess was incorrect. This data can be
corrected by manipulating the [ihm.System object](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System) directly, at the end of the script, immediately before we call ProtocolOutput's `flush` method. We will look at a few examples in this section.

## Cross-linker type<a id="xltype"></a>

For cross-linking experiments, the mmCIF file contains a description of the cross-linking reagent used. This information is not in the CSV file or the Python script. ProtocolOutput guesses the name of the reagent using the
`label` passed to the PMI [CrossLinkingMassSpectrometryRestraint](https://integrativemodeling.org/2.12.0/doc/ref/classIMP_1_1pmi_1_1restraints_1_1crosslinking_1_1CrossLinkingMassSpectrometryRestraint.html),
as this label is often the name of the cross-linker. However, in this case it is not (we used the labels `Trnka` and `Chen` respectively for the datasets from [Al Burlingame's lab](http://www.mcponline.org/content/13/2/420.long) and
[Juri Rappsilber's lab](http://emboj.embopress.org/content/29/4/717)). We can correct this by looking up the publications to determine that the [DSS](https://en.wikipedia.org/wiki/Disuccinimidyl_suberate) and [BS3](https://en.wikipedia.org/wiki/Bissulfosuccinimidyl_suberate) cross-linkers were used, respectively, and using the python-ihm API to set the correct [cross-linker type](https://python-ihm.readthedocs.io/en/latest/cross_linkers.html#module-ihm.cross_linkers) for each [cross-linking restraint](https://python-ihm.readthedocs.io/en/latest/restraint.html#ihm.restraint.CrossLinkRestraint) in the [list of all restraints](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.restraints):

In [ ]:
import ihm.cross_linkers
import ihm.location
import ihm.model
import RMF
import IMP.rmf

# Fix linker types to match those actually used (DSS and BS3)
trnka, chen = [r for r in po.system.restraints if hasattr(r, 'linker')]
trnka.linker = ihm.cross_linkers.dss
chen.linker = ihm.cross_linkers.bs3

## Correct number of output models<a id="fixnummodel"></a>

ProtocolOutput correctly notes that we ran Monte Carlo to generate 20000 frames. However, in many modeling scenarios the modeling script is run multiple times on a compute cluster to generate several independent trajectories which are
then combined. ProtocolOutput cannot know whether this happened. However, it is straightforward to use the python-ihm API to manually change the number of output models to match that reported in the publication:

In [ ]:
# Correct number of output models to account for multiple runs
protocol = po.system.orphan_protocols[-1]
protocol.steps[-1].num_models_end = 200000

## Add model coordinates<a id="addcoords"></a>

The current mmCIF file contains all of the input data, and notes that Monte Carlo was used to generate frames, but doesn't actually store any of those coordinates in the file. Ultimately this information will be added automatically
by PMI model analysis and validation scripts, but for the time being any information about clustering, localization densities, and final models needs to be added to the file using the python-ihm API:

In [ ]:
# Get last protocol in the file
protocol = po.system.orphan_protocols[-1]
# State that we filtered the 200000 frames down to one cluster of
# 100 models:
analysis = ihm.analysis.Analysis()
protocol.analyses.append(analysis)
analysis.steps.append(ihm.analysis.ClusterStep(
                      feature='RMSD', num_models_begin=200000,
                      num_models_end=100))

python-ihm allows for models to be placed into groups ([ihm.model.ModelGroup](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.ModelGroup)) so we can make such a group for the cluster. These groups are in turn grouped in an [ihm.model.State](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.State), a distinct state of the system (e.g. open/closed form, or reactant/product in a chemical reaction). Finally, states can themselves be grouped into [ihm.model.StateGroup](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.StateGroup) objects. All of these groups act like regular Python lists. In this case we have only one state, so we just add our model group to that:

In [ ]:
mg = ihm.model.ModelGroup(name="Cluster 0")

# Add to last state
po.system.state_groups[-1][-1].append(mg)

Next, we can describe the cluster using the [ihm.model.Ensemble](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.Ensemble) class, noting that it was generated from our clustering:

In [ ]:
e = ihm.model.Ensemble(model_group=mg,
                       num_models=100,
                       post_process=analysis.steps[-1],
                       name="Cluster 0")
po.system.ensembles.append(e)

To add a model to the file, we create an [ihm.model.Model](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.Model) object, add atoms or coarse-grained objects to it, and then add that to the previously-created [ModelGroup](https://python-ihm.readthedocs.io/en/latest/model.html#ihm.model.ModelGroup). ProtocolOutput provides a convenience function `add_model` which does this, converting the current IMP structure to python-ihm (we just need to load an IMP structure first, e.g. from an RMF file using [IMP.rmf.load_frame](https://integrativemodeling.org/2.12.0/doc/ref/namespaceIMP_1_1rmf.html#abc15209a0aef0e859c99ce00659ebdfb)).

In [ ]:
# Add the model from RMF (let's say it's frame 42 from the output RMF file)
rh = RMF.open_rmf_file_read_only('output/rmfs/0.rmf3')
IMP.rmf.link_hierarchies(rh, [root_hier])
IMP.rmf.load_frame(rh, RMF.FrameID(42))
del rh
model = po.add_model(e.model_group)

If localization densities are available for the cluster, we can link to those too. For example, to add the density of the Rpb4 subunit for the entire cluster:

In [ ]:
# Look up the ihm.AsymUnit corresponding to a PMI component name
asym = po.asym_units['Rpb4.0']
# Add path to a local output file
loc = ihm.location.OutputFileLocation('output/cluster0.Rpb4.mrc')
den = ihm.model.LocalizationDensity(file=loc, asym_unit=asym)
# Add to ensemble
e.densities.append(den)

## Replace local links with DOIs<a id="adddois"></a>

[Recall from earlier](#linking) that ProtocolOutput adds local paths to the cross-link CSV files and the Python script itself, which won't be visible to end users. The simplest way to solve this issue is to put some or all of the
files in zipfiles and upload them to a service that provides a DOI, such as [Zenodo](https://zenodo.org/) or [FigShare](https://figshare.com/), and then link to that DOI. We can do this by creating an
[ihm.location.Repository](https://python-ihm.readthedocs.io/en/latest/location.html#ihm.location.Repository) object for each zipfile, and then using the [update_locations_in_repositories](https://python-ihm.readthedocs.io/en/latest/main.html#ihm.System.update_locations_in_repositories) function to map local paths to files in the zipfile at the DOI:

In [ ]:
# Replace local links with DOIs
repo = ihm.location.Repository(doi="10.5281/zenodo.2598760", root="../..",
                  top_directory="salilab-imp_deposition_tutorial-1ad5919",
                  url="https://zenodo.org/record/2598760/files/salilab/"
                      "imp_deposition_tutorial-v0.2.zip")
po.system.update_locations_in_repositories([repo])

This assumes that the zipfile at the given DOI and URL was created by archiving all files under the top directory of the [GitHub repository](https://github.com/salilab/imp_deposition_tutorial) (`../..` relative to the directory containing the script). (It was actually created automatically [by Zenodo](https://zenodo.org) when a new release of this tutorial was created on GitHub. Zenodo puts all the files in the zipfile under a uniquely-named directory, so we need to set `top_directory` to match.)

Thus the cross-link file which was previously linked with the local path `../data/polii_xlinks.csv` can now be found by downloading `imp_deposition_tutorial-v0.2.zip` and extracting `salilab-imp_deposition_tutorial-1ad5919/rnapolii/data/polii_xlinks.csv` from it.

# Output<a id="output"></a>

We can now save the entire protocol to an mmCIF file (or to [BinaryCIF](https://github.com/dsehnal/BinaryCIF), if we have the Python [msgpack](https://pypi.org/project/msgpack/) package installed) using the [ihm.dumper.write](https://python-ihm.readthedocs.io/en/latest/dumper.html#ihm.dumper.write) function:

In [ ]:
po.finalize()
with open('rnapolii.cif', 'w') as fh:
    ihm.dumper.write(fh, [po.system])

# Visualization<a id="visualization"></a>

## ChimeraX<a id="chimerax"></a>

mmCIF files can be viewed in many viewers. However, most viewers do not yet support the integrative modeling extensions, and so may only show the atomic parts of the model (if any). Integrative models can be viewed in
[ChimeraX](https://www.rbvi.ucsf.edu/chimerax/) - be sure to use a recent nightly build, and open the file from the ChimeraX command line using the `format ihm` option, e.g. `open rnapolii.cif format ihm`.
[VMD](http://www.ks.uiuc.edu/Research/vmd/) is also reportedly working on support in their forthcoming 1.9.4 release.

An example view of the deposition, as rendered by ChimeraX, is shown below. The EM map is shown as a mesh, the DSS cross-links as green dashed lines, and the BS3 cross-links as blue dashed lines.

<img src="images/chimerax.png" width="700px" title="ChimeraX visualization of rnapoliii mmCIF file" />

## Web browser<a id="browser"></a>

mmCIF files can also be viewed in a web browser, using [Mol* Viewer](https://molstar.org/viewer).

## Plain text<a id="plain"></a>

mmCIF files are also just plain text files, and can be viewed in any text editor (for example to check for errors, particularly for categories that ChimeraX doesn't support yet). Most of the data is stored as simple tables
(look for the `loop_` keyword in the file). For example, the coordinates of the coarse-grained beads are stored in the [_ihm_sphere_obj_site table](http://mmcif.wwpdb.org/dictionaries/mmcif_ihm.dic/Categories/ihm_sphere_obj_site.html),
the start of which in `rnapolii.cif` looks like:

```
loop_
_ihm_sphere_obj_site.ordinal_id
_ihm_sphere_obj_site.entity_id
_ihm_sphere_obj_site.seq_id_begin
_ihm_sphere_obj_site.seq_id_end
_ihm_sphere_obj_site.asym_id
_ihm_sphere_obj_site.Cartn_x
_ihm_sphere_obj_site.Cartn_y
_ihm_sphere_obj_site.Cartn_z
_ihm_sphere_obj_site.object_radius
_ihm_sphere_obj_site.rmsf
_ihm_sphere_obj_site.model_id
1 1 1 1 A 158.180 180.876 96.861 3.068 . 1
2 1 2 2 A 105.913 135.689 154.058 2.888 . 1
3 1 3 3 A 104.305 132.810 156.027 2.273 . 1
4 1 4 4 A 101.027 134.743 156.557 3.008 . 1
```

This simply states that a sphere representing residue 1 in chain A is centered
at (158.180, 180.876, 96.861) and has radius 3.068, and so on.